In [1]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_set'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

# Create a list to store DataFrames
all_dfs = []

for file in filenames:
  try:
    # Use 'on_bad_lines' instead of 'error_bad_lines'
    # df = pd.read_csv(file, on_bad_lines='skip', delimiter=",")  # Skip bad lines
    df = pd.read_csv(file)

    # 정상화
    df.columns=['index', 'data']
    df = df.set_index('index')
    df['spam'] = 1
    df['data'] = df['data'].str.replace('ifg@','')

    # 정규식을 사용해 한글, 영어, 숫자 데이터만 남겨두기
    df['data'] = df['data'].str.replace(r'[^가-힣a-zA-Z0-9]', ' ', regex=True)

    # Dask Dataframe 생성
    # Move ddf creation after modifying the Pandas DataFrame
    ddf = dd.from_pandas(df, npartitions=4) # Create ddf after modifications


    # Append the current DataFrame to the list
    result = ddf.compute()
    all_dfs.append(result)

  except pd.errors.ParserError as e:
    print(f"Error reading file {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_dfs, ignore_index=True)

In [2]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['data'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

62716
62716


In [3]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('data')
print(len(raw))

62716
60773


In [5]:
 # spam 여부 판단 컬럼 타입 변경
raw['spam'] = raw['spam'].fillna(1).astype(int)
raw.dtypes

data    object
spam     int32
dtype: object

In [6]:
raw['data'] = raw['data'].str.replace('Web발신', '')
raw['data'] = raw['data'].str.replace('광고', '')

In [8]:
raw.head()

,data,spam
0,님 작전주 에 당첨되셨습니다 이 문자는 분뒤 삭제됩니다 방...,1
1,국제발신 처음 치킨cupon새벽 방문 만hum luck 쩜컴CD hb ...,1
2,국제발신 이오스 엔지엔 에볼루정답쥐조작쥐후불제결과로보답합니다백프로입니다엔트리안내 ...,1
3,국제발신 회 오 리 커피값 시 시회 오 리가 아니면 사 칭사칭 먹튀에 주의...,1
4,국외발신 새고객 원 출금해 가세요 십만원샌즈 W Ri계열 새...,1


In [7]:
raw.to_csv('C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_data.csv', index=False)